In [2]:
import numpy as np
import pandas as pd
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from datasets import Dataset
import pickle
from utils.helper import read_py150k_code, read_file_to_string
import regex as re

fname_prefix = "/data/users/team2_capstone/code-style-probing/"

In [3]:

csv_fname = "bq_data_outlier_no_class_no_super.csv"


In [11]:
feat_name = "no_class_no_super"

In [13]:
# #uncommented prep
# combined_df = pd.read_csv(fname_prefix + "data/labeled_code/bq_uncommented_outlier.csv")
# combined_df = combined_df[combined_df['uncommented_content'].notnull()]
# #test_df = combined_df.query("content != uncommented_content")
# #display(combined_df)
# #display(test_df)
# dataset = Dataset.from_pandas(combined_df).train_test_split(test_size=0.2)
# print (1)

1


In [14]:
#class prep
#class_df= pd.read_csv(fname_prefix + "data/labeled_code/bq_data_outlier_no_class.csv")
#class_df['uncommented_content'] = combined_df['uncommented_content']
#class_df = class_df[class_df['uncommented_content'].notnull()]
#class_df = class_df[class_df['no_class_content'].notnull()]
#display (class_df)
#dataset = Dataset.from_pandas(class_df).train_test_split(test_size=0.2)
print (1)

1


In [15]:
# """combined_df = pd.read_csv(fname_prefix + "data/labeled_code/bq_uncommented_outlier.csv")
# docstring_df = pd.read_csv(fname_prefix + "data/labeled_code/bq_no_docstring_outlier.csv")
# docstring_df['uncommented_content'] = combined_df['uncommented_content']
# docstring_df = docstring_df[docstring_df['uncommented_content'].notnull()]
# docstring_df = docstring_df[docstring_df['no_docstring_content'].notnull()]
# #display (docstring_df)
# dataset = Dataset.from_pandas(docstring_df).train_test_split(test_size=0.2)"""

'combined_df = pd.read_csv(fname_prefix + "data/labeled_code/bq_uncommented_outlier.csv")\ndocstring_df = pd.read_csv(fname_prefix + "data/labeled_code/bq_no_docstring_outlier.csv")\ndocstring_df[\'uncommented_content\'] = combined_df[\'uncommented_content\']\ndocstring_df = docstring_df[docstring_df[\'uncommented_content\'].notnull()]\ndocstring_df = docstring_df[docstring_df[\'no_docstring_content\'].notnull()]\n#display (docstring_df)\ndataset = Dataset.from_pandas(docstring_df).train_test_split(test_size=0.2)'

In [4]:


df = pd.read_csv(fname_prefix + f"data/labeled_code/{csv_fname}")
display(df)

,Unnamed: 0,Unnamed: 0.1,repository,filepath,forks,issue_events,stars,parse_error,line_count,comment_count,...,snake_case_class,upper_camel_case_method,other_case_method,upper_case_class,lower_camel_case_class,upper_case_method,file,labels,content,no_class_content
0,0,0,jbkalmbach/kbmod,analysis/rawGenerated/batchGenerate.py,1,32,1,1,51.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,jbkalmbach/kbmodanalysis/rawGenerated/batchGen...,-1,from kbmodpy import kbmod as kb\nimport rawSea...,NaN
1,1,1,pymedusa/SickRage,medusa/providers/torrent/rss/rsstorrent.py,22,763,27,1,170.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,pymedusa/SickRagemedusa/providers/torrent/rss/...,-1,"# coding=utf-8\n\n""""""Provider code for RSS Tor...",NaN
2,2,2,ipdata/python,ipdata/test_cli.py,6,15,16,1,135.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,ipdata/pythonipdata/test_cli.py,-1,import multiprocessing\nimport sys\nimport uni...,NaN
3,3,3,demisto/content,Packs/AnsibleLinux/Integrations/AnsibleACME/An...,1328,52,797,1,59.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,demisto/contentPacks/AnsibleLinux/Integrations...,-1,import traceback\nimport ssh_agent_setup\nimpo...,NaN
4,4,4,hidat/audio_pipeline,review_parser/mb_release.py,1,26,4,1,76.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,hidat/audio_pipelinereview_parser/mb_release.py,-1,import json\nimport os\nimport os.path as path...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875091,875091,87927,operepo/ope,libs/gluon/packages/dal/pydal/adapters/sqlite.py,8,138,9,1,279.0,14.0,...,NaN,NaN,10.0,NaN,NaN,2.0,operepo/opelibs/gluon/packages/dal/pydal/adapt...,-1,# -*- coding: utf-8 -*-\nimport copy\nimport d...,NaN
875092,875092,87928,DOV-Vlaanderen/pydov,setup.py,17,287,31,1,66.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,DOV-Vlaanderen/pydovsetup.py,-1,#!/usr/bin/env python\n# -*- coding: utf-8 -*-...,NaN
875093,875093,87929,slogan621/tscharts,apps/xrayuploader/xrayuploader.py,8,97,16,1,502.0,20.0,...,NaN,NaN,3.0,NaN,NaN,NaN,slogan621/tschartsapps/xrayuploader/xrayupload...,-1,#(C) Copyright Syd Logan 2020\n#(C) Copyright ...,NaN
875094,875094,87930,seasonstar/bibi,application/models/order/logistic.py,365,12,1016,1,434.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,seasonstar/bibiapplication/models/order/logist...,-1,# -*- coding: utf-8 -*-\nimport math\nimport r...,NaN


In [5]:
uncomment_df = pd.read_csv(fname_prefix + f"data/labeled_code/bq_uncommented_outlier.csv")
display(uncomment_df)

,Unnamed: 0,Unnamed: 0.1,repository,filepath,forks,issue_events,stars,parse_error,line_count,comment_count,...,snake_case_class,upper_camel_case_method,other_case_method,upper_case_class,lower_camel_case_class,upper_case_method,file,labels,content,uncommented_content
0,0,0,jbkalmbach/kbmod,analysis/rawGenerated/batchGenerate.py,1,32,1,1,51.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,jbkalmbach/kbmodanalysis/rawGenerated/batchGen...,-1,from kbmodpy import kbmod as kb\nimport rawSea...,from kbmodpy import kbmod as kb\nimport rawSea...
1,1,1,pymedusa/SickRage,medusa/providers/torrent/rss/rsstorrent.py,22,763,27,1,170.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,pymedusa/SickRagemedusa/providers/torrent/rss/...,-1,"# coding=utf-8\n\n""""""Provider code for RSS Tor...","""""""Provider code for RSS Torrents.""""""\nfrom __..."
2,2,2,ipdata/python,ipdata/test_cli.py,6,15,16,1,135.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,ipdata/pythonipdata/test_cli.py,-1,import multiprocessing\nimport sys\nimport uni...,import multiprocessing\nimport sys\nimport uni...
3,3,3,demisto/content,Packs/AnsibleLinux/Integrations/AnsibleACME/An...,1328,52,797,1,59.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,demisto/contentPacks/AnsibleLinux/Integrations...,-1,import traceback\nimport ssh_agent_setup\nimpo...,import traceback\nimport ssh_agent_setup\nimpo...
4,4,4,hidat/audio_pipeline,review_parser/mb_release.py,1,26,4,1,76.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,hidat/audio_pipelinereview_parser/mb_release.py,-1,import json\nimport os\nimport os.path as path...,import json\nimport os\nimport os.path as path...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875091,875091,87927,operepo/ope,libs/gluon/packages/dal/pydal/adapters/sqlite.py,8,138,9,1,279.0,14.0,...,NaN,NaN,10.0,NaN,NaN,2.0,operepo/opelibs/gluon/packages/dal/pydal/adapt...,-1,# -*- coding: utf-8 -*-\nimport copy\nimport d...,import copy\nimport datetime\nimport locale\ni...
875092,875092,87928,DOV-Vlaanderen/pydov,setup.py,17,287,31,1,66.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,DOV-Vlaanderen/pydovsetup.py,-1,#!/usr/bin/env python\n# -*- coding: utf-8 -*-...,"""""""The setup script.""""""\nfrom setuptools impor..."
875093,875093,87929,slogan621/tscharts,apps/xrayuploader/xrayuploader.py,8,97,16,1,502.0,20.0,...,NaN,NaN,3.0,NaN,NaN,NaN,slogan621/tschartsapps/xrayuploader/xrayupload...,-1,#(C) Copyright Syd Logan 2020\n#(C) Copyright ...,"import getopt, sys\nimport json\nimport time\n..."
875094,875094,87930,seasonstar/bibi,application/models/order/logistic.py,365,12,1016,1,434.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,seasonstar/bibiapplication/models/order/logist...,-1,# -*- coding: utf-8 -*-\nimport math\nimport r...,import math\nimport random\nimport datetime\ni...


In [6]:
target_column = "no_class_content"

In [7]:
df["uncommented_content"] = uncomment_df["uncommented_content"]

In [9]:


df = df[df['uncommented_content'].notnull()]
df = df[df[target_column].notnull()]
df = df.query(f"uncommented_content != {target_column}")
display (df)
dataset = Dataset.from_pandas(df).train_test_split(test_size=0.1)

,Unnamed: 0,Unnamed: 0.1,repository,filepath,forks,issue_events,stars,parse_error,line_count,comment_count,...,upper_camel_case_method,other_case_method,upper_case_class,lower_camel_case_class,upper_case_method,file,labels,content,no_class_content,uncommented_content
8,8,8,osrg/gobgp,test/scenario_test/bgp_zebra_test.py,687,1066,2972,1,312.0,40.0,...,NaN,NaN,NaN,NaN,NaN,osrg/gobgptest/scenario_test/bgp_zebra_test.py,7,# Copyright (C) 2015 Nippon Telegraph and Tele...,\nimport sys\nimport time\nimport unittest\nim...,import sys\nimport time\nimport unittest\nimpo...
32,32,32,SpamExperts/SpamPAD,tests/functional/test_rules/test_tflags.py,6,112,10,1,92.0,0.0,...,NaN,NaN,NaN,NaN,NaN,SpamExperts/SpamPADtests/functional/test_rules...,-1,"""""""Functional tests for tflags option""""""\n\nfr...",\n'Functional tests for tflags option'\nfrom _...,"""""""Functional tests for tflags option""""""\nfrom..."
70,70,70,swevm/scaleio-py,scaleiopy/api/scaleio/mapping/volume.py,24,20,17,1,57.0,3.0,...,NaN,NaN,NaN,NaN,NaN,swevm/scaleio-pyscaleiopy/api/scaleio/mapping/...,-1,# Imports\n\n# Project imports\nfrom scaleiopy...,\nfrom scaleiopy.api.scaleio.mapping.sio_gener...,from scaleiopy.api.scaleio.mapping.sio_generic...
74,74,74,techbliss/Python_editor,7.5/Add to Python 2 PyQt5 folder/uic/Compiler/...,61,18,230,1,101.0,52.0,...,NaN,NaN,NaN,NaN,NaN,techbliss/Python_editor7.5/Add to Python 2 PyQ...,-1,##############################################...,"\nfrom .misc import Literal, moduleMember\n' P...","from .misc import Literal, moduleMember\n\ncla..."
108,108,108,Guts/DicoGIS,dicogis/georeaders/geo_infos_generic.py,11,47,18,1,180.0,38.0,...,NaN,NaN,NaN,NaN,NaN,Guts/DicoGISdicogis/georeaders/geo_infos_gener...,-1,#! python3 # noqa: E265\n\n\n# ##############...,\nimport logging\nfrom collections import Orde...,import logging\nfrom collections import Ordere...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874931,874931,87767,tensorflow/agents,tf_agents/bandits/environments/stationary_stoc...,859,982,3052,1,134.0,21.0,...,NaN,NaN,NaN,NaN,NaN,tensorflow/agentstf_agents/bandits/environment...,-1,# coding=utf-8\n# Copyright 2020 The TF-Agents...,\n'Class implementation of Stationary Stochast...,"""""""Class implementation of Stationary Stochast..."
874952,874952,87788,Pal3love/otRebuilder,Package/otRebuilder/Dep/fontTools/feaLib/parse...,4,2,45,1,1496.0,58.0,...,NaN,NaN,NaN,NaN,NaN,Pal3love/otRebuilderPackage/otRebuilder/Dep/fo...,-1,"from __future__ import print_function, divisio...","\nfrom __future__ import print_function, divis...","from __future__ import print_function, divisio..."
875013,875013,87849,JayTeeGeezy/pyracing,pyracing/test/horses.py,2,78,1,1,79.0,0.0,...,NaN,NaN,NaN,NaN,NaN,JayTeeGeezy/pyracingpyracing/test/horses.py,-1,from .common import *\n\n\nclass GetHistorical...,\nfrom .common import *\n\n@classmethod\ndef s...,from .common import *\n\nclass GetHistoricalHo...
875060,875060,87896,kadrlica/obztak,obztak/seeing.py,3,10,4,1,273.0,32.0,...,NaN,NaN,NaN,NaN,NaN,kadrlica/obztakobztak/seeing.py,-1,"#!/usr/bin/env python\n""""""\nGeneric python scr...",\n'\nGeneric python script.\n'\n__author__ = '...,"""""""\nGeneric python script.\n""""""\n__author__ =..."


In [10]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration, PLBartTokenizer

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')
# example =dataset['train']['uncommented_content']
#label = tokenizer(example, truncation=True)
#print (len(label['input_ids']))
def tokenization(example):
    return_dict = tokenizer(example[target_column], padding='max_length', truncation=True)
    #print ((example['content'][10]))
    labels = tokenizer(example['uncommented_content'], padding='max_length', truncation=True).input_ids
    return_dict['labels'] = labels
    #print (return_dict.keys())
    
    return return_dict
train_dataset = dataset["train"].map(tokenization, batched=True)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
train_dataset.format['type']

test_dataset = dataset["test"].map(tokenization, batched=True)
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.format['type']

loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/vocab.json from cache at /data/users/cting3/.cache/huggingface/transformers/1e2aacf615bc83f25a9d748eccb762b335eee01a29ab7a8db9b8e86cc851d489.9a48c5abf25554713c6513ab01066e53569b9a2da0d6189715951cf7c6288805
loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/merges.txt from cache at /data/users/cting3/.cache/huggingface/transformers/7eaa9b856402f05e8fdd452951872ecd3c2692ea9abb86b7ab62b07e3bc5f7de.7179059568f1a130b0a79e4bac71f38545207cab0ec45ce82ca09afadb2649a3
loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/added_tokens.json from cache at /data/users/cting3/.cache/huggingface/transformers/a3e93db547e41cdd21f01826d07c5679e111b02d8e969c607611c30a6acbe191.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b
loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/special_tokens_map.json from cache at /data/users/cting3/.cache/huggingface/transformer

  0%|          | 0/35 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

'torch'

In [12]:
print ((train_dataset[0]))
train_dataset.save_to_disk(fname_prefix + f"datasets/codet5_train_{feat_name}_bq_padded.hf")
test_dataset.save_to_disk(fname_prefix + f"datasets/codet5_test_{feat_name}_bq_padded.hf")
#print (combined_df[combined_df['content'] ==  "File Error"])
#combined_df.to_csv(fname_prefix + 'data/labeled_code/full_data.csv')

{'labels': tensor([    1,  5666,  2836,  3813,   203,  2080,  2527,  1930, 15777,    16,
        20650,   203,   203,  1106, 11432,    30,   203,   565, 11829,   273,
          374,   203,   565,  7733, 25061,   273,   404,   203,   565,   348,
         1099,    58,  2602,   273,   576,   203,   565, 13839,   273,   890,
          203,   565,  7910,    44,  3871,   273,  1059,   203,   565,   611,
         1157,  1974,  1738,   273,  1381,   203,   565,  7527,   273,  1666,
          203,   565,   777,   273,   444,  3816,  9826,    16,  7733, 25061,
           16,   348,  1099,    58,  2602,    16, 13839,    16,  7910,    44,
         3871,    16,   611,  1157,  1974,  1738,    16,  7527,  5717,   203,
          203,   565,  1652, 14855,    12,   701,  4672,   203,  3639,   309,
          609,   422,   296,  6102,  4278,   203,  5411,   327, 11829,   203,
         3639,  1327,   609,   422,   296, 10772,  4278,   203,  5411,   327,
         7733, 25061,   203,  3639,  1327,   609,   4

In [13]:
train_dataset

Dataset({
    features: ['Unnamed: 0', 'Unnamed: 0.1', 'repository', 'filepath', 'forks', 'issue_events', 'stars', 'parse_error', 'line_count', 'comment_count', 'comment_avg_len', 'comment_density', 'id_total', 'lower_case', 'id_total_var', 'lower_case_var', 'snake_case', 'snake_case_var', 'lower_camel_case', 'lower_camel_case_var', 'snake_case_ratio', 'snake_case_var_ratio', 'snake_case_class_ratio', 'snake_case_method_ratio', 'lower_camel_case_ratio', 'lower_camel_case_var_ratio', 'lower_camel_case_class_ratio', 'lower_camel_case_method_ratio', 'upper_camel_case_ratio', 'upper_camel_case_var_ratio', 'upper_camel_case_class_ratio', 'upper_camel_case_method_ratio', 'lower_case_ratio', 'lower_case_var_ratio', 'lower_case_class_ratio', 'lower_case_method_ratio', 'upper_case_ratio', 'upper_case_var_ratio', 'upper_case_class_ratio', 'upper_case_method_ratio', 'other_case_ratio', 'other_case_var_ratio', 'other_case_class_ratio', 'other_case_method_ratio', 'func_decorators_avg', 'func_async_